In [86]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [127]:
#최적화 이전 버전
#http://www.menupan.com/restaurant/bestrest/bestrest.asp?pt=rt&areacode=cs208
# 전체 내용 데이터를 크롤링하기

# parser 종류 : html.parser => html5lib 사용
# 순위, 상호, 업종, 지역, a 태그 url(음식점) 출력
html = 'http://www.menupan.com/restaurant/bestrest/bestrest.asp'
param = '?pt=rt&areacode=cs208&page='

# response = requests.get(html)
# print(response.status_code)
#del str
for i in range(1,5):
    url = html + param + str(i)
    response = requests.get(url)
    if(response.status_code == 200):
      soup = BeautifulSoup(response.text, 'html5lib')

      #순위
      rank_set = soup.select('body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul')
      for n in rank_set:
        rank = n.select(  'li > div > p.numTop' )
        rank_nonTop =  n.select(  'li > div > p.rankNum' )
        rank = rank + rank_nonTop
        rank = [x.text for x in rank]
        # print(rank)


      #상호
      #name a 태그의 텍스트 부분만 가져오기
      name_set = soup.select('body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul')
      # body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul > li:nth-child(9) > p.listName > span > a
      for n in name_set:
        name = n.select( 'li > p.listName > span > a' )
        name = [x.text for x in name]
        # print(name)

      #업종
      mtype_set = soup.select('body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul')
      # > li:nth-child(1) > p.listType
      for x in mtype_set:
        mtype = x.select('li > p.listType')
      mtype = [x.text for x in mtype]
      # print(mtype)

      #지역
      region = soup.select('body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul')
      # > li:nth-child(1) > p.listArea
      for x in region:
        region = x.select('li > p.listArea')
      region = [x.text for x in region]
      # print(region)

      #a태그
      a_tag = soup.select('body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul ')
      #> li:nth-child(1) > p.listName > span > a
      for x in a_tag:
        a_tag = x.select('li > p.listName > span > a')
      a_tag = [x.attrs['href'] for x in a_tag]
      # print(a_tag)

for x in range(len(rank)):
  print(rank[x], name[x], mtype[x], region[x], a_tag[x])




In [136]:
#위 코드 최적화 버전
#http://www.menupan.com/restaurant/bestrest/bestrest.asp?pt=rt&areacode=cs208
# 전체 내용 데이터를 크롤링하기

# parser 종류 : html.parser => html5lib 사용
# 순위, 상호, 업종, 지역, a 태그 url(음식점) 출력
html = 'http://www.menupan.com/restaurant/bestrest/bestrest.asp'
param = '?pt=rt&areacode=cs208&page='

response = requests.get(html)
# print(response.status_code)
#del str

# 가져온 데이터들을 list 로 나타내는 작업

# [
#   {'순위': ~~~, '상호':~~, '업종':~~, }
#   {'순위': ~~~, '상호':~~, '업종':~~, }
#  ]
menulist = []

# 단, url에 대한 내용은 별개의 list에 넣기
urllist = []
for i in range(1,5):
    url = html + param + str(i)
    response = requests.get(url)
    if(response.status_code == 200):
      soup = BeautifulSoup(response.content, 'html5lib')

      #최적화 버전
      data_set = soup.select('body > div > div.container > div.bestRest > div.ranking > div.rankingList > ul > li')
      for n in data_set:

        #쉼표가 or 의 역할을 함
        rank = n.select(  '.numTop, .rankNum' )

        # rank_nonTop =  n.select(  'div > p.rankNum' )
        # rank = rank + rank_nonTop
        rank = [x.text for x in rank]
        # print(rank)

        name = n.select( 'p.listName > span > a' )
        name = [x.text for x in name]
        # print(name)

        mtype = n.select('p.listType')
        mtype = [x.text for x in mtype]

        region = n.select('p.listArea')
        region = [x.text for x in region]

        a_tag = n.select('p.listName > span > a')
        a_tag = [x.attrs['href'] for x in a_tag]

        menulist.append({'순위': rank[0], '상호': name[0], '업종': mtype[0], '지역': region[0]})
        urllist.append(a_tag[0])
      # print(rank)
      # print(name)
      # print(mtype)
      # print(region)
      # print(a_tag)

# for x in menulist:
#     # 현재 항목 `x`의 인덱스를 `menulist`에서 찾습니다.
#     index = menulist.index(x)
#     # 항목과 그에 맞는 URL을 출력합니다.
#     print(x, urllist[index])

for x,y in zip(menulist,urllist):
  print(x, y)



{'순위': '1', '상호': '모박사', '업종': '한식', '지역': '경기 안성시 대덕면'} /restaurant/onepage.asp?acode=H492290
{'순위': '2', '상호': '서일농원', '업종': '한식', '지역': '경기 안성시 일죽면'} /restaurant/onepage.asp?acode=H492288
{'순위': '3', '상호': '안성마춤갤러리', '업종': '한식', '지역': '경기 안성시 공도읍'} /restaurant/onepage.asp?acode=H492291
{'순위': '4', '상호': '안성장터국밥', '업종': '한식', '지역': '경기 안성시 도기동'} /restaurant/onepage.asp?acode=H492294
{'순위': '5', '상호': '팔도식당', '업종': '한식', '지역': '경기 안성시 연지동'} /restaurant/onepage.asp?acode=H493141
{'순위': '6', '상호': '강건너빼리', '업종': '한식', '지역': '경기 안성시 금광면'} /restaurant/onepage.asp?acode=H493112
{'순위': '7', '상호': '안성맞춤한우촌', '업종': '한식', '지역': '경기 안성시 삼죽면'} /restaurant/onepage.asp?acode=H493302
{'순위': '8', '상호': '별난매운탕', '업종': '한식', '지역': '경기 안성시 양성면'} /restaurant/onepage.asp?acode=H493115
{'순위': '9', '상호': '조개야', '업종': '한식', '지역': '경기 평택시 비전동'} /restaurant/onepage.asp?acode=H493719
{'순위': '10', '상호': '김가부대찌개', '업종': '한식', '지역': '경기 안성시 공도읍'} /restaurant/onepage.asp?acode=H493135
{'순위': '11', '상호': '돌담집', '업종

In [110]:
# 위의 데이터들을 list 로 나타내는 작업

# [
#   {'순위': ~~~, '상호':~~, '업종':~~, }
#   {'순위': ~~~, '상호':~~, '업종':~~, }
#  ]

# menulist = []
# for x in range(len(rank)):
#   menulist.append({'순위': rank[x], '상호': name[x], '업종': mtype[x], '지역': region[x]})

# # print(menulist)
# # 단, url에 대한 내용은 별개의 list에 넣기
# urllist = []
# for x in range(len(rank)):
#     urllist.append(a_tag[x])
# # print(urllist, '\n')




In [139]:
# menuList를 가지고 순위를 index로 하는 데이터프레임을 하나 생성
df = pd.DataFrame(menulist)
# df = df.set_index('순위')
df.set_index('순위', inplace=True)
df

,상호,업종,지역
순위,,,
1,모박사,한식,경기 안성시 대덕면
2,서일농원,한식,경기 안성시 일죽면
3,안성마춤갤러리,한식,경기 안성시 공도읍
4,안성장터국밥,한식,경기 안성시 도기동
5,팔도식당,한식,경기 안성시 연지동
6,강건너빼리,한식,경기 안성시 금광면
7,안성맞춤한우촌,한식,경기 안성시 삼죽면
8,별난매운탕,한식,경기 안성시 양성면
9,조개야,한식,경기 평택시 비전동


In [140]:
# http://www.menupan.com 이 기본 주소 , 여기에 urlList에 담아놓은 주소값 붙여서 요청
# 상세 페이지로 넘어갔을 때 나오는 전화 번호, 상세 주소를 가져오고
# 기존에 만들어놓은 DataFrame에 붙이기 ( + '주소' 열은 삭제 )
base_add = 'http://www.menupan.com'
full_str = []
tell_list = []
address_list = []
for x in urllist:
  full_str.append(base_add + x)
  response2 = requests.get(base_add + x)
  if(response2.status_code == 200):
    soup2 = BeautifulSoup(response2.content, 'html5lib')
    #body > center > div.WrapMain > div.mainArea01 > div.areaBasic > dl.restTel > dd 전화번호
    #body > center > div.WrapMain > div.mainArea01 > div.areaBasic > dl.restAdd > dd.add2 > a 상세 주소
    tell = soup2.select('body > center > div.WrapMain > div.mainArea01 > div.areaBasic > dl.restTel > dd')
    tell = [x.text for x in tell]
    print(tell[0])

    address = soup2.select('body > center > div.WrapMain > div.mainArea01 > div.areaBasic > dl.restAdd > dd.add2 > a')
    address = [x.text for x in address]
    print(address[0])

    tell_list.append(tell[0])
    address_list.append(address[0])

df = df.drop('지역', axis=1)
#또는 df.rop(columns='지역',inplace=True)
df['전화번호'] = tell_list
df['주소'] = address_list
df


(031) 676-1508
경기 안성시 대덕면 서동대로 4653
(031) 673-3171
경기 안성시 일죽면 금일로 332-17
(031) 658-3960
경기 안성시 공도읍 서동대로 4469
(031) 674-9494
경기 안성시 안성맞춤대로 960
(031) 674-2111
경기 안성시 혜산로 12
(031) 671-0007
경기 안성시 금광면 가협길 179-150
(031) 673-5550
경기 안성시 삼죽면 신미1길 126
(031) 673-2393
경기 안성시 양성면 만세로 735
(031) 618-3555
경기 평택시 문화촌로19번길 7-3
(031) 692-1566
경기 안성시 공도읍 공도1로 20
(031) 671-7909
경기 안성시 서운면 바우덕이로 316
(031) 676-3007
경기 안성시 태평무길 31
(031) 673-9981
경기 안성시 양성면 도곡길 34-15
(031) 678-6700
경기 안성시 삼죽면 국사봉로 641-12
(031) 675-1805
경기 안성시 보개면 남사당로 151
(031) 676-0069
경기 안성시 금광면 동신곡길 58
(031) 656-4987
경기 안성시 원곡면 칠곡호수길 37
(031) 672-4728
경기 안성시 양성면 만세로 667
(031) 675-4029
경기 안성시 낙원길 43
(031) 671-0511
경기 안성시 죽산면 칠장로 16
(031) 674-1235
경기 안성시 개내교1길 2
(031) 675-4703
경기 안성시 중앙로371번길 54


,상호,업종,전화번호,주소
순위,,,,
1,모박사,한식,(031) 676-1508,경기 안성시 대덕면 서동대로 4653
2,서일농원,한식,(031) 673-3171,경기 안성시 일죽면 금일로 332-17
3,안성마춤갤러리,한식,(031) 658-3960,경기 안성시 공도읍 서동대로 4469
4,안성장터국밥,한식,(031) 674-9494,경기 안성시 안성맞춤대로 960
5,팔도식당,한식,(031) 674-2111,경기 안성시 혜산로 12
6,강건너빼리,한식,(031) 671-0007,경기 안성시 금광면 가협길 179-150
7,안성맞춤한우촌,한식,(031) 673-5550,경기 안성시 삼죽면 신미1길 126
8,별난매운탕,한식,(031) 673-2393,경기 안성시 양성면 만세로 735
9,조개야,한식,(031) 618-3555,경기 평택시 문화촌로19번길 7-3


In [143]:
#업종이 한식인 가게의 정보만 출력
# Pandas에서 Series 내의 특정 문자열 찾기 : str.contains
df[df['업종']=='한식']

,상호,업종,전화번호,주소
순위,,,,
1,모박사,한식,(031) 676-1508,경기 안성시 대덕면 서동대로 4653
2,서일농원,한식,(031) 673-3171,경기 안성시 일죽면 금일로 332-17
3,안성마춤갤러리,한식,(031) 658-3960,경기 안성시 공도읍 서동대로 4469
4,안성장터국밥,한식,(031) 674-9494,경기 안성시 안성맞춤대로 960
5,팔도식당,한식,(031) 674-2111,경기 안성시 혜산로 12
6,강건너빼리,한식,(031) 671-0007,경기 안성시 금광면 가협길 179-150
7,안성맞춤한우촌,한식,(031) 673-5550,경기 안성시 삼죽면 신미1길 126
8,별난매운탕,한식,(031) 673-2393,경기 안성시 양성면 만세로 735
9,조개야,한식,(031) 618-3555,경기 평택시 문화촌로19번길 7-3


In [145]:
# 특정 구에 있는 가게의 정보만 출력
# Pandas에서 Series 내의 특정 문자열 찾기 : str.contains
df[df['주소'].str.contains('금광면')]

,상호,업종,전화번호,주소
순위,,,,
6,강건너빼리,한식,(031) 671-0007,경기 안성시 금광면 가협길 179-150
16,물고기자리 안성점,한식,(031) 676-0069,경기 안성시 금광면 동신곡길 58
